### Load dependencies

In [56]:
#imports.... Run this each time after restarting the Kernel
#!pip install -e git+https://github.com/ibm-watson-data-lab/pixiedust@david-gateway-branch#egg=pixiedust
#!pip install --user --upgrade pixiedust
#!pip install watson_developer_cloud
import pixiedust
import json
import pandas
import datetime
import watson_developer_cloud as watson
import json
import time
import re
import pyspark
import sys
import pandas as pd
from botocore.client import Config
import ibm_boto3


#Notes from meeting with David Taieb
### Github Repo, Notebooks there and users create notebook from url.
### Add good markdown comments and documentation in Notebook
### Notebook 1, Fetch from COS(with audio files, we provide), run audio analysis, put transcript in their COS()
### notebook 2, TA, NLU, NLC => give data frame
### dashboard (UI), code is there and they can experiment and 
### publish it as webapp with pixie gateway, just show how to do it
### .



## Create Watson service and copy paste credentials

Instructions to go to IBM cloud and creating the service

In [57]:
# The code was removed by DSX for sharing.

## Set-up Object storage

In [58]:
# The code was removed by DSX for sharing.

### NLU

- process text, chunk it
- analyze transcript

In [65]:
#NLU

from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding.features import (
    v1 as Features)

natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2017-02-27',
    username=username_NLU,
    password=password_NLU)

chunk_size=30

def chunk_transcript(transcript, chunk_size):
    transcript = transcript.split(' ')
    return [ transcript[i:i+chunk_size] for i in range(0, len(transcript), chunk_size) ] # chunking data
    

def process_text(text):
    transcript=''
    for sentence in json.loads(text)['results']:
        transcript = transcript + sentence['alternatives'][0]['transcript'] # concatenate sentences
    transcript = chunk_transcript(transcript, chunk_size) # chunk the transcript
    return transcript


def analyze_transcript(features, file_name):
    streaming_body = client.get_object(Bucket = credentials['BUCKET'], Key=file_name)['Body']
    transcript=streaming_body.read().decode("utf-8")
    nlu_analysis={}
    for chunk in process_text(transcript):
        chunk = ' '.join(chunk)
        print(chunk)
        nlu_analysis[chunk] = natural_language_understanding.analyze(features, chunk, return_analyzed_text=True)
    res=client.put_object(Bucket = credentials['BUCKET'], Key=file_name[0].split('_')[0]+'_tone', Body= json.dumps(nlu_analysis))
    return nlu_analysis


In [66]:
text_files = ['sample1-addresschange-positive_text.json', 'sample2-address-negative_text.json', 'sample3-shirt-return-weather-chitchat_text.json', 'sample4-angryblender-sportschitchat-recovery_text.json', 'sample5-calibration-toneandcontext_text.json'] # we add audio files to COS pre-conference
features = {"concepts":{},"entities":{},"keywords":{},"categories":{},"emotion":{},"sentiment":{},"semantic_roles":{} }

analysis = analyze_transcript(features, text_files[0])
print(analysis)

good morning can you give me some help I'd like to change my address please my name is Ryan Smith I am from Sacramento California that's right my phone number
is five five five one two one two yes that's me my old address is number one two three oak street my new address is five six seven pine street
yes and the zip is nine zero two one zero yep that's right now the phone number stays the same that's right I would like to keep all the options
of said no other changes the only thing that I want to change is the address yes that's right yep very good yes thank you so much for help it
thanks have a good day bye bye 
{"of said no other changes the only thing that I want to change is the address yes that's right yep very good yes thank you so much for help it": {'entities': [], 'categories': [{'score': 0.226511, 'label': '/shopping/resources/contests and freebies'}, {'score': 0.16996, 'label': '/law, govt and politics/legal issues/legislation/tax laws'}, {'score': 0.169616, 'label': '/busin